In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize, odr

https://stackoverflow.com/questions/78432247/fit-a-model-with-multiple-inputs-multiple-outputs-multiple-parameters-and-cov

In [2]:
def model(XY, a, b, c):
    x, y = XY
    u = a * x + b * y
    v = a * x * y - c
    return u, v

In [3]:
# TRUE MODEL FIELD
a, b, c = -1, 7, 11 # True model parameters
x, y = np.meshgrid(np.linspace(-5, 5, 10), np.linspace(-5, 5, 10))
u, v = model((x, y), a, b, c)

In [4]:
target = np.hstack([np.ravel(u), np.ravel(v)])

In [5]:

# CREATE DATA
data_size = 100
np.random.seed(0)
x_data, y_data = np.random.uniform(-5, 5, data_size), np.random.uniform(-5, 5, data_size)
u_data_expected, v_data_expected = model((x_data, y_data), a, b, c)
noise_field = [np.random.normal(0, 5, data_size), np.random.normal(0, 5, data_size)]
u_data_real, v_data_real = u_data_expected + noise_field[0], v_data_expected + noise_field[1]

In [23]:
# CREATE DATA UNCERTAINTIES AND CORRELATIONS
# Input data (positions) uncertainties and correlations:
x_err, y_err = np.random.normal(0, 0.1, data_size), np.random.normal(0, 0.1, data_size)
x_y_corr = np.random.uniform(-1, 1, data_size)
cov_matrices_xy = np.array([[[x_err[i]**2, x_err[i]*y_err[i]*x_y_corr[i]],
                             [x_err[i]*y_err[i]*x_y_corr[i], y_err[i]**2]] for i in range(data_size)])

# Output data (vectors) uncertainties and correlations:
u_err, v_err = np.random.normal(0, 0.1, data_size), np.random.normal(0, 0.1, data_size)
u_v_corr = np.random.uniform(-1, 1, data_size)
cov_matrices_uv = np.array([[[u_err[i]**2, u_err[i]*v_err[i]*u_v_corr[i]],
                             [u_err[i]*v_err[i]*u_v_corr[i], v_err[i]**2]] for i in range(data_size)])

In [55]:
Cx = (X.T - X.mean(axis=1)).T @ (X.T - X.mean(axis=1))
Cu = (U.T - U.mean(axis=1)).T @ (U.T - U.mean(axis=1))

In [6]:
target = np.hstack([np.ravel(u_data_real), np.ravel(v_data_real)])

In [7]:
def fitter(p, x):
    u = p[0] * x[0] + p[1] * x[1]
    v = p[0] * x[0] * x[1] - p[2]
    return np.array([u, v])

In [8]:
X = np.array([x_data.ravel(), y_data.ravel()])

In [9]:
U = np.array([u_data_real.ravel(), v_data_real.ravel()])

In [10]:
X.shape

(2, 100)

In [11]:
U.shape

(2, 100)

In [12]:
model = odr.Model(fitter)

In [57]:
data = odr.RealData(X, U, covx=Cx, covy=Cu)

In [58]:
fit = odr.ODR(data, model, beta0=[1., 1., 1.])

In [59]:
result = fit.run()

In [60]:
result.pprint()

Beta: [-1.2483959   7.10308325 11.59454377]
Beta Std Error: [0.10658793 0.42941014 0.58865156]
Beta Covariance: [[  6.49905209  -8.76721063   3.66030481]
 [ -8.76721063 105.48205575 -17.93062359]
 [  3.66030481 -17.93062359 198.22141981]]
Residual Variance: 0.001748098943636363
Inverse Condition #: 0.18760356570269432
Reason(s) for Halting:
  Sum of squares convergence
